# Office 365 Exchange
## Run query, save results as Excel, and send as attachment

This notebook will take in three input parameters: first, it will accept a comma-delimited (,) list of email addresses, and the query ID of the associated saved query in the workspace, and the sending mailbox address. Using these values, it loads the Databricks SDK, and reads the query text from the file and executes it as a dataframe operation.

With the resulting dataframe, it will then convert it to a Pandas dataframe, and save the result as Excel, using local node storage (for reasons: https://learn.microsoft.com/en-us/azure/databricks/files/write-data#where-should-i-write-temporary-files-on-azure-databricks)

Once saved, a new authentication request will be made using a service principal. Secret scopes values need to be created as follows (feel free to change the scope name and/or secret names as needed):
<br />
<br />
```
Secret Scope name: dbx-mail-secrets

dbx-mail-tenant-id: Tenant ID 
dbx-mail-client-id: Client ID of the associated service principal
dbx-mail-secret: Client secret
```
<br />
Make sure your service principal has the approporate API persmissions (details: https://o365.github.io/python-o365/latest/getting_started.html#authentication). This example does not use delegated permissions, so scopes are not required.

**NOTE:** This methods uses a "Direct Send" method of sending email through Office 365, which does NOT support external (aka non-domain) users! If you need to send exteral emails, check out the basic SMTP notebook instead. However, this method is preferred for internal only communications for organizations that use Office 365 / Exchange

More details here: https://learn.microsoft.com/en-us/exchange/mail-flow-best-practices/how-to-set-up-a-multifunction-device-or-application-to-send-email-using-microsoft-365-or-office-365 

In [0]:
%pip install o365 openpyxl xlsxwriter

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("query_id", "", "Query ID")
dbutils.widgets.text("email_recipients", "", "Email Recipients (comma separated)")
dbutils.widgets.text("from_mailbox", "", "From Email Address")

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

In [0]:
requested_query = dbutils.widgets.get("query_id")
query = w.queries.get(requested_query)
query_obj = query.as_dict()
query_text = query_obj["query_text"]
query_name = query_obj["display_name"]

### DEBUG: What query actually ran?

In [0]:
print(query_text)

In [0]:
query_results_df = spark.sql(query_text)

In [0]:
query_results_pandas_df = query_results_df.toPandas()

In [0]:
import datetime
report_time = datetime.datetime.now()
report_time_formatted = report_time.strftime("%Y-%m-%d_%H:%M")
report_time_formatted


In [0]:
file_name = "/local_disk0/tmp/{0}_{1}.xlsx".format(query_name.replace(" ","_"), report_time_formatted)

In [0]:
import pandas as pd
writer = pd.ExcelWriter(file_name) 
query_results_pandas_df.to_excel(writer, sheet_name=query_name, index=False, na_rep='NaN')

for column in query_results_pandas_df:
    column_length = max(query_results_pandas_df[column].astype(str).map(len).max(), len(column))
    col_idx = query_results_pandas_df.columns.get_loc(column)
    writer.sheets[query_name].set_column(col_idx, col_idx, column_length)

writer.close()

### Set up email with O365 Python Library

This notebook usese the popular open source Python library, O365: https://github.com/O365/python-o365

This library takes care of all the requied modern authentication and GraphAPI calls required to properly set up a connection to Office 365 Exchange. You're more than welcome to write your own if you prefer not to use this method.

In [0]:
tenant_id = dbutils.secrets.get(scope = "dbx-mail-secerts", key = "dbx-mail-tenant-id")
client_id = dbutils.secrets.get(scope = "dbx-mail-secerts", key = "dbx-mail-client-id")
client_secret = dbutils.secrets.get(scope = "dbx-mail-secerts", key = "dbx-mail-secret")


In [0]:
from O365 import Account

credentials = (client_id, client_secret)

account = Account(credentials, auth_flow_type='credentials', tenant_id=tenant_id)
if account.authenticate():
   print('Authenticated!')

message = account.new_message(resource=dbutils.widgets.get("from_mailbox"))
to_list = dbutils.widgets.get("email_recipients").split(",")
for email in to_list:
    message.to.add(email)
message.subject = 'Databricks Report: {0}'.format(query_name)
message.sender.address = dbutils.widgets.get("from_mailbox")
message.body = 'Please find your automated Databricks-geneated report attached. This report was generated on {0}'.format(report_time_formatted)
message.attachments.add(file_name)
message.send()